#### Note:

Make sure that you have these two scripts running at the same time while running this

ark regsitry start

python sim_node.py

In [215]:
import time
import numpy as np
from arktypes import joint_group_command_t, task_space_command_t, joint_state_t
from arktypes.utils import pack, unpack
from ark.client.comm_infrastructure.instance_node import InstanceNode

ROBOT_NAME = "arkbot"
SIM = False

class HarkControllerNode(InstanceNode):
    def __init__(self):
        super().__init__("arkbot_controller")
        suffix = "/sim" if SIM else ""
        base = f"{ROBOT_NAME}"

        # pubs
        self.joint_group_command = self.create_publisher(f"{base}/joint_group_command{suffix}", joint_group_command_t)
        self.task_space_command  = self.create_publisher(f"{base}/cartesian_command{suffix}", task_space_command_t)

        # subs
        self.state = self.create_listener(f"{base}/joint_states{suffix}", joint_state_t)

node = HarkControllerNode()

[WARNING] [15:07:22.465354] - No global configuration provided. Using default system configuration.
[INFO] [15:07:22.469452] - Service: Successfully registered '__DEFAULT_SERVICE/GetInfo/arkbot_controller_f5dda695-c398-4cb9-9817-149905b04684' with registry.
[INFO] [15:07:22.471563] - Service: Successfully registered '__DEFAULT_SERVICE/SuspendNode/arkbot_controller_f5dda695-c398-4cb9-9817-149905b04684' with registry.
[INFO] [15:07:22.476646] - Service: Successfully registered '__DEFAULT_SERVICE/RestartNode/arkbot_controller_f5dda695-c398-4cb9-9817-149905b04684' with registry.
[ERROR] [15:07:22.477356] - Couldn't load config for other 'arkbot_controller'
[OK] [15:07:22.477931] - setup publisher arkbot/joint_group_command[joint_group_command_t]
[OK] [15:07:22.478344] - setup publisher arkbot/cartesian_command[task_space_command_t]
[OK] [15:07:22.478880] - subscribed to arkbot/joint_states[joint_state_t]


None


---

# Joint Group Control

In [ ]:
all_cmd = [
    0.0,   # Revolute 1 (continuous)
     0.0,   # Revolute 2 (rad)
     0.0,   # Revolute 4 (rad)
     0.00,   # Revolute 5 (continuous)
     0.00,   # Revolute 6 (rad)
     0.000   # Slider 7 position in meters (range 0.0 .. 0.0425)
]
node.joint_group_command.publish(pack.joint_group_command(all_cmd, "all"))

In [ ]:
# save the position offset, click this once when the robot is in the zero position
pos_off = unpack.joint_state(node.state.get())


In [7]:
# GRIPPER (velocity mode): single value for Slider 7 velocity (m/s)
gripper_vel_open  = [ +0.010 ]   # open slowly
gripper_vel_close = [ -0.010 ]   # close slowly
node.joint_group_command.publish(pack.joint_group_command(gripper_vel_open, "gripper"))

In [8]:
# ARM (position mode): 6 values, Revolute 1..6
arm_cmd = [-0.30, 0.50, 0.30, -1.00, 0.10, 0.80]
node.joint_group_command.publish(pack.joint_group_command(arm_cmd, "arm"))


---

# Task Space Control

In [7]:
xyz_command = np.array([0.3, 0.4, 0.8])
quaternion_command = np.array([1, 0.0, 0.0, 0.0]) # xyz-w
gripper = 1.0 # 0.0 close, 1.0 open

controller.task_space_command.publish(
    pack.task_space_command("all", xyz_command, quaternion_command, gripper))

---


# Joint States

In [222]:
unpack.joint_state(node.state.get())


({'seq': 0, 'stamp': {'sec': 0, 'nsec': 0}, 'frame_id': ''},
 ['Revolute 1',
  'Revolute 2',
  'Revolute 3',
  'Revolute 4',
  'Revolute 5',
  'Revolute 6',
  'Slider 7'],
 array([0.2493571 , 0.55973255, 0.02641856, 0.67341757, 3.20755383,
        0.79000011, 4.48229186]),
 array([0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0.]))